In [ ]:
%tensorflow_version 2.x
import tensorflow as tf

t=tf.zeros([2,2,2,2]) #vytvoří tensor, zaplněn nulami s dimenzemi v závorkách
t=tf.reshape(t,[2,-1]) #přeskupí vybraný tensor, na jiný tensor, s jinými dimenzemi, které si určíme, můžeme si také dimenzi dopočítat zadáním -1
print(t)

In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals

import numpy as np #better arrays in python, lepší práce s multidimenzionálními poli
import pandas as pd #data analytics tool, lepší manipulace s daty, dokáže například cut outnout column
import matplotlib.pyplot as plt #vizualizace tabulek a grafů
from IPython.display import clear_output #jen pro tenhle notebook
from six.moves import urllib

import tensorflow.compat.v2.feature_column as fc #

import tensorflow as tf


# Load dataset.
dftrain = pd.read_csv('https://storage.googleapis.com/tf-datasets/titanic/train.csv') # training data
dfeval = pd.read_csv('https://storage.googleapis.com/tf-datasets/titanic/eval.csv') # testing data
#print(dftrain.head()) #vypíše nám prvních 5 záznamů a hlavičky dat
#print(dftrain["age"]) #vypíše sloupec age
y_train = dftrain.pop('survived') #vyjme z dat sloupec survived a uloží ho do y_train
y_eval = dfeval.pop('survived')
#print(dftrain.loc[0])#vypíše 1. řádek dat
#dftrain.describe() je metoda vhodná pro statistiky, dává nám údaje o datech jako vážený průměr, počet, min, max atd.
#dftrain.shape nám vrátí počet řádků a počet sloupců dat
#print(dftrain.age.hist(bins=20)) #vytvoří histogram, který reprezentuje data
#print(dftrain.sex.value_counts().plot(kind='barh')) vytvoří graf zobrazujicí počet pohlaví
#pd.concat([dftrain,y_train], axis=1).groupby('sex').survived.mean().plot(kind='barh').set_xlabel('% survive')# graf, jenž nám zobrazí jakou má které pohlaví šanci na přežití

CATEGORICAL_COLUMNS=['sex','n_siblings_spouses','parch','class','deck','embark_town','alone']
NUMERIC_COLUMNS=['age','fare']

feature_columns=[]
for feature_name in CATEGORICAL_COLUMNS:
  vocabulary=dftrain[feature_name].unique() #projede všechny řádky a zjistí každou unikátní hodnotu
  feature_columns.append(tf.feature_column.categorical_column_with_vocabulary_list(feature_name,vocabulary))

for feature_name in NUMERIC_COLUMNS:
  feature_columns.append(tf.feature_column.numeric_column(feature_name,dtype=tf.float32))

#print(feature_columns)

#následujicí kód nám poslouží abychom získali dataset object, kterému náš model bude rozumět
#epochs=kolikrát uvidí náš model data
#batch_size=Počet řádků, které uvidí každým průchodem, aby nedošlo k přehlcení paměti

def make_input_fn(data_df, label_df, num_epochs=10, shuffle=True, batch_size=32):
  def input_function():#vnitřní funkce, která bude vrácena
    ds=tf.data.Dataset.from_tensor_slices((dict(data_df),label_df))# vytvoří tf.data.Dataset object a naplní ho daty 
    if shuffle: #slouží k tomu aby náš model nahlížel data i zjiné perspektivy
      ds=ds.shuffle(1000)#promíchá naše data
    ds=ds.batch(batch_size).repeat(num_epochs) # rozdělí dataset do 32 řádkových batchů a opakuje tolikrát kolik máme epochů
    return ds #vrátí nám batch datasetu
  return input_function # vrátí funkci pro náš objekt

train_input_fn=make_input_fn(dftrain,y_train) # here we will call the input_function that was returned to us to get a dataset object we can feed to the model
eval_input_fn = make_input_fn(dfeval, y_eval, num_epochs=1, shuffle=False)

linear_est=tf.estimator.LinearClassifier(feature_columns=feature_columns)

linear_est.train(train_input_fn)#train
result=linear_est.evaluate(eval_input_fn)#výsledky toho, jak si model vedl

clear_output()#vyčistí konzoli
print(result['accuracy'])#vypíše nám výsledek, ale result obsahuje i například průměrný loss, jenž zde je 0.5, protože je to binární problém
#print(result)

#následujicí kód zkouší predikovat na základě vybraných dat

result=list(linear_est.predict(eval_input_fn))
print(dfeval.loc[5])
print(y_eval.loc[5])
print(result[5]['probabilities'][1])#index 1 nám zobrazí šanci, že přežil




In [ ]:
#Klasifikace 1:55:03
#https://www.youtube.com/watch?v=tPYj3fFJGjk